In [121]:
import numpy as np
import heapq
import math
import random
from typing import Callable, TypeVar
from collections import Counter, deque, defaultdict
import itertools
from functools import cmp_to_key, cache, lru_cache
import regex as re
from intervaltree import Interval, IntervalTree
from concurrent.futures import ThreadPoolExecutor
from sortedcontainers import SortedDict
import z3
import networkx as nx

np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

T = TypeVar('T')

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

processors = {
  'int_list': lambda x: [int(y) for y in x.split()],
  'int_string': lambda x: [int(y) for y in x]
}

# TODO: replace with nx
def search(start, get_neighbors, end_condition=lambda _, __: False, method='dfs', permutations=False):
    q, visited = [(0, start)] if method == 'dijkstra' else deque([(0, start)]), {}
    while q:
        if method == 'dijkstra':
            distance, current = heapq.heappop(q)
        else:
            distance, current = q.popleft() if method=='dfs' else q.pop()
        if end_condition(current, distance):
            visited[current] = distance
            return visited, current
        if current in visited and not permutations:
            continue
        for node in get_neighbors(current, distance):
            if method == 'dijkstra':
                heapq.heappush(q, node)
            else:
                q.append((distance+1, node))
        visited[current] = distance
    return visited, None

def binary_search(fn, low, high, target):
    if high >= low:
        mid = (high + low) // 2
        if fn(mid) == target:
            return mid
        return binary_search(fn, low, mid - 1, target) if fn(mid) > target else binary_search(fn, mid + 1, high, target)
    return -1

def sum_series(start, stop):
    n = (stop - start)
    sum = start + stop
    return n * sum // 2

def debug_array(arr, coords, p=True, v='X'):
    arr = arr.copy()
    for i in coords:
        arr[*i] = v
    if p:
        print(arr)
    return arr

def flatten(xss):
    return [x for xs in xss for x in xs]

def yx(n):
    return int(n.imag), int(n.real)
def icoord(y, x):
    return int(x)+1j*int(y)

In [96]:

def day1():
    loc1, loc2 = zip(*data(1, processors['int_list']))
    part1 = sum(abs(x[0]-x[1]) for x in zip(sorted(loc1), sorted(loc2)))
    counts = Counter(loc2)
    part2 = sum(x*counts[x] for x in loc1)
    return part1, part2

day1()

(1941353, 22539317)

In [97]:
def day2():
    def check_safe(report):
        ascending = sorted(report)
        diffs = np.diff(ascending)
        return max(diffs) <= 3 and min(diffs) >= 1 and (
            report == ascending or
            report == list(reversed(ascending))
        )

    def check_safe_damp(report):
        if check_safe(report):
            return 1, 1
        for damped in itertools.combinations(report, len(report)-1):
            if check_safe(list(damped)):
                return 0, 1
        return 0, 0

    reports = data(2, processors['int_list'])
    safe = np.array((0,0))
    for report in reports:
        safe += check_safe_damp(report)
    return safe

day2()

array([356, 413])

In [98]:
def day3():
    def mul_strings(s):
        x, y = s.split(',')
        return int(x)*int(y)

    instructions = ''.join(data(3))
    matches = list(re.finditer(r'mul\((\d+,\d+)\)', instructions))
    conds = list(re.finditer(r"don't\(\).+?do\(\)", instructions))
    donts = IntervalTree([Interval(*cond.span()) for cond in conds])
    result = sum([mul_strings(mul[1]) * (1 if not donts[mul.span()[0]] else 1j) for mul in matches])
    return int(result.real+result.imag), int(result.real)

day3()

(182780583, 90772405)

In [99]:
def day4():
    grid = np.array(data(4, lambda x: np.array(list(x))))
    ymax, xmax = grid.shape

    def find_target_occurences(target):
        occurences = set()

        def get_neighbors(current, distance):
            row, col = yx(current)
            target_letter = target[distance]
            if grid[row, col] != target_letter:
                return
            if distance == len(target)-1:
                total.add(current)
                return
            for v in [1, -1, 1j, -1j, 1+1j, 1-1j, -1+1j, -1-1j]:
                new = current + v
                y, x = yx(new)
                if not (y >= 0 and x >= 0 and y < ymax and x < xmax):
                    continue
                yield new
        
        for j in range(ymax):
            for i in range(xmax):
                total = set()
                coordinate = i+1j*j
                search(coordinate, get_neighbors)
                for end in total:
                    occurences.add((coordinate, end))
        return occurences

    def find_diags(hits, l):
        centers = Counter()
        for start, end in hits:
            distance = end-start
            if abs(distance.real) == l and abs(distance.imag) == l:
                center = start + distance/2
                centers[center] += 1
        return centers
    
    def find_straights(hits, target):
        rev = target[::-1]
        l = len(target)
        td = l-1
        for start, end in hits:
            j, i = yx(start)
            d = end-start
            if (
                (d.real == td and not d.imag and ''.join(grid[j, i:i+l]) == target)
                or (d.real == -td and not d.imag and ''.join(grid[j, i-td:i+1]) == rev)
                or (not d.real and d.imag == td and ''.join(grid[j:j+l, i]) == target)
                or (not d.real and d.imag == -td and ''.join(grid[j-td:j+1, i]) == rev)
            ):
                yield start

    target = 'XMAS'
    matches = find_target_occurences(target)
    part1 = sum(find_diags(matches, len(target)-1).values()) + len(list(find_straights(matches, target)))

    centers = find_diags(find_target_occurences(target[1:]), len(target)-2)
    part2 = sum([1 if centers[x] == 2 else 0 for x in centers])

    return (part1, part2)

day4()

(2599, 1948)

In [86]:
def day5():
    text = data(5)
    split = text.index('')
    lists = [tuple(map(int, x.split(','))) for x in text[split+1:]]

    parents = defaultdict(lambda: set())
    for x in text[:split]:
        parent, child = tuple(map(int, x.split('|')))
        parents[child].add(parent)

    def check_illegal(nums):
        illegal = set()
        for num in nums:
            if num in illegal:
                return True
            illegal.update(parents[num])

    def compare(a, b):
        if a in parents[b]:
            return 1
        elif b in parents[a]:
            return -1
        return -1 if a < b else 1

    part1, part2 = 0, 0
    for nums in lists:
        if not check_illegal(nums):
            part1 += nums[len(nums)//2]
        else:
            part2 += sorted(nums, key=cmp_to_key(compare))[len(nums)//2]
            
    return part1, part2

day5()

(6041, 4884)

In [89]:
def day6():
    grid = np.array(data(6, list))
    ymax, xmax = grid.shape
    start = np.argwhere(grid == '^')[0]
    grid[*start] = '.'
    turns = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    
    def run_guard(obstacle=(-1, -1)):
        directions, v = itertools.cycle(turns), turns[-1]
        y, x = int(start[0])-v[1], int(start[1])-v[0]
        visited, states = set(), set()
        while True:
            ny, nx = y+v[1], x + v[0]
            if (ny, nx, v) in states:
                return True, states
            elif (nx < 0 or ny < 0 or nx >= xmax or ny >= ymax):
                return False, visited
            elif grid[ny, nx] != '.' or (ny, nx) == obstacle:
                v = next(directions)
                continue
            y, x = ny, nx
            visited.add((y, x))
            states.add((y, x, v))

    _, original = run_guard()
    part1 = len(original)
    with ThreadPoolExecutor() as tpe:
        part2 = sum([r[0] for r in tpe.map(lambda x: run_guard(x), original)])
    return part1, part2

day6()

(4374, 1705)

In [90]:
def day7():
    equations = data(7, lambda x: [int(n) for n in re.split(r' |: ', x)])
    
    def check(n, acc, arr):
        if not arr:
            return n == acc
        elif acc > n:
            return False
        x, tail = arr[0], arr[1:]
        return (check(n, int(f'{acc}{x}'), tail) or
                check(n, acc*x, tail) or
                check(n, acc+x, tail))

    result = 0
    for eq in equations:
        if check(eq[0], eq[1], tuple(eq[2:])):
            result += eq[0]
    print(result)


day7()

20928985450275


In [ ]:
def day7():
    equations, flag = data(7, lambda x: [int(n) for n in re.split(r' |: ', x)]), False
    def check(n, acc, arr):
        if not arr:
            return n if n == acc else 0
        elif acc > n:
            return
        x, tail = arr[0], arr[1:]
        return ((flag and check(n, int(f'{acc}{x}'), tail)) or
                check(n, acc*x, tail) or
                check(n, acc+x, tail))
    part1, flag = sum([check(eq[0], eq[1], tuple(eq[2:])) for eq in equations]), True
    with ThreadPoolExecutor() as tpe:
        part2 = sum(tpe.map(lambda eq: check(eq[0], eq[1], tuple(eq[2:])), equations))
    return part1, part2

day7()

(7710205485870, 20928985450275)

In [ ]:
def day8():
    grid = np.array(data(8, list))
    ymax, xmax = grid.shape
    def is_inside(y, x):
        return y >= 0 and y < ymax and x >= 0 and x < xmax
    def get_nodes(start, diff, sign=1):
        node = start.copy()
        while is_inside(*node):
            yield tuple(node)
            node -= sign*diff

    part1, part2 = set(), set()
    points = {i:np.argwhere(grid==i) for i in np.unique(grid) if i != '.'}
    for antennae in points:
        combos = itertools.combinations(points[antennae], 2)
        for combo in combos:
            diff = combo[1]-combo[0]
            for y, x in (combo[0]-diff, combo[1]+diff):
                if is_inside(y, x):
                    part1.add((y, x))
            for (y, x) in [*get_nodes(combo[0], diff), *get_nodes(combo[1], diff, -1)]:
                part2.add((y, x))

    return len(part1), len(part2)

day8()

(261, 898)

In [ ]:
def day9():
    disk_map = data(9)[0]
    files = [int(x) for x in disk_map[0::2]]
    buffers = [int(x) for x in disk_map[1::2]]

    def part1(buffers):
        p = 0 # Current pointer location
        buffer = 0 # Current buffer space available
        fid = 0 # Current file number from left to right
        checksum = 0 # Result
        for n in range(len(files)-1, -1, -1):
            req = files[n]
            while req > buffer:
                # fill buffer with rightmost file
                checksum += sum_series(p, p+buffer)*n
                p += buffer
                req -= buffer
                # add current file to checksum
                fsize = files[fid]
                checksum += sum_series(p, p+fsize)*fid
                p += fsize
                fid += 1
                if fid >= n:
                    checksum += sum_series(p, p+req)*fid
                    return checksum
                # get next buffer
                buffer = next(buffers)
            checksum += sum_series(p, p+req)*n
            p += req
            buffer -= req

    def part2(buffers):
        p = 0
        starts = {}
        slots = SortedDict()
        for i, v in enumerate(files):
            starts[i] = p
            p += v
            try:
                buffer = next(buffers)
                if buffer:
                    slots[p] = buffer
                    p += buffer
            except StopIteration:
                continue

        def find_slot(size): # O(n)
            for index, value in slots.items():
                if value >= size:
                    return index
            return -1

        checksum = 0
        for n in range(len(files)-1, -1, -1):
            req = files[n]
            p = find_slot(req)
            if p != -1 and p < starts[n]:
                buffer = slots[p]
                del slots[p]
                if (rem := buffer-req):
                    slots[p+req] = rem
            else:
                p = starts[n]
            checksum += sum_series(p, p+req)*n
        return checksum
    
    return part1(iter(buffers)), part2(iter(buffers))

day9()

(6349606724455, 6376648986651)

In [ ]:
def day10():
    grid = np.array(data(10, processors['int_string']))
    ymax, xmax = grid.shape
    starts = np.argwhere(grid==0)

    def get_neighbors(current, distance):
        for v in (1, -1, 1j, -1j):
            new = current + v
            y, x = yx(new)
            if not (y >= 0 and x >= 0 and y < ymax and x < xmax):
                continue
            if grid[y, x] == distance + 1:
                yield new

    def trail_end(current, distance):
        if distance == 9:
            trailheads.add(current)

    def rating_end(current, distance):
        if distance == 9:
            ratings.append(current)

    part1, part2 = 0, 0
    for y, x in starts:
        trailheads, ratings = set(), []
        search(x+1j*y, get_neighbors, trail_end)
        search(x+1j*y, get_neighbors, rating_end, permutations=True)
        part1 += len(trailheads)
        part2 += len(ratings)
    return part1, part2

day10()

(737, 1619)

In [ ]:
def day11():
    stones = data(11, processors['int_list'])[0]

    def next_stones(s):
        if not s:
            return (1, )
        ss = str(s)
        half, rem = divmod(len(ss), 2)
        if not rem:
            return int(ss[:half + rem]), int(ss[half + rem:])
        return (s*2024, )
    
    @cache
    def run_stones(s, i):
        if not i:
            return [s]
        result = flatten([run_stones(stone, i-1) for stone in next_stones(s)])
        return result

    @cache
    def stone_count(s, i):
        if not i:
            return 1
        stones = run_stones(s, 1)
        result = 0
        for stone in stones:
            result += stone_count(stone, i-1)
        return result

    part1 = sum(stone_count(s, 25) for s in stones)
    part2 = sum(stone_count(s, 75) for s in stones)
    return part1, part2

day11()

(186424, 219838428124832)

In [ ]:
def day12():
    grid = np.array(data(12, list))
    ymax, xmax = grid.shape
    
    def corner_count(n, y, x):
        size = len(n)
        if not size:
            return 4
        elif size == 1:
            return 2
        elif size == 2: 
            b, a = yx(n[0])
            j, i = yx(n[1])
            if j != b and i != a: # L shape
                return 1 if grid[j, a] == grid[b, i] else 2
        else:
            ys, xs = list(zip(*[yx(node) for node in n])) # Check diagonals
            bot, top, left, right = min(ys), max(ys), min(xs), max(xs)
            return len([i for i in [(bot, left), (bot, right), (top, left), (top, right)] if grid[i] != grid[y, x]])
        return 0

    def get_neighbors(current, _):
        target = yx(current)
        neighbors = []
        for v in (1, -1, 1j, -1j):
            new = current + v
            y, x = yx(new)
            if not (y >= 0 and x >= 0 and y < ymax and x < xmax):
                continue
            if grid[y, x] == grid[*target]:
                neighbors.append(new)
                perimeters[current] -= 1
        corners[current] = corner_count(neighbors, *target)
        return neighbors

    part1, part2 = 0, 0
    visited = set()
    perimeters, corners = defaultdict(lambda: 4), {}
    for j in range(ymax):
        for i in range(xmax):
            coordinate = i + 1j*j
            if coordinate in visited:
                continue
            region, _ = search(coordinate, get_neighbors)
            visited.update(region)
            part1 += len(region)*sum(perimeters[r] for r in region.keys())
            part2 += len(region)*sum(corners[r] for r in region.keys())
    return part1, part2

day12()

(1473276, 901100)

In [ ]:
def day13():
    lines = data(13)
    groups = [lines[i:i+3] for i in range(0, len(lines), 4)]

    def parse_coords(text, prize=False):
        r = r'Prize: X=(\d+), Y=(\d+)' if prize else r'Button .: X\+(\d+), Y\+(\d+)'
        matches = re.findall(r, text)[0]
        return int(matches[0]), int(matches[1])

    def solve(a, b, prize):
        solver = z3.Optimize()
        x = z3.Int('x')
        y = z3.Int('y')
        solver.minimize(3*x+y)
        solver.add(a[0]*x + b[0]*y == prize[0])
        solver.add(a[1]*x + b[1]*y == prize[1])
        
        solver.check()
        result = solver.model()
        return 3*result[x].as_long() + result[y].as_long() if result else 0

    part1, part2 = 0, 0
    for group in groups:    
        a = parse_coords(group[0])
        b = parse_coords(group[1])
        prize = parse_coords(group[2], True)
        part1 += solve(a, b, prize)
        part2 += solve(a, b, (prize[0]+10000000000000, prize[1]+10000000000000))
    return part1, part2

day13()

(36758, 76358113886726)

In [ ]:
def day14(wide, tall):
    robots = data(14, lambda line: [int(x) for x in re.findall(r'p=(\d+),(\d+) v=(-?\d+),(-?\d+)', line)[0]])
    
    def get_quadrants(xs):
        lr, tb = (wide-1)/2, (tall-1)/2
        quadrants = Counter()
        for y, x in xs:
            if x < lr and y < tb:
                quadrants[1] += 1
            elif x < lr and y > tb:
                quadrants[2] += 1
            elif x > lr and y < tb:
                quadrants[3] += 1
            elif x > lr and y > tb:
                quadrants[4] += 1
        return quadrants

    def run_robot(robot, n):
        x, y, vx, vy = robot
        fx, fy = (x+n*vx)%wide, (y+n*vy)%tall
        return fy, fx

    def get_danger(n):
        quadrants = get_quadrants([run_robot(robot, n) for robot in robots])
        danger = math.prod((quadrants[x] for x in range(1, 5)))
        return danger, n

    _, n = min(get_danger(n) for n in range(wide*tall))
    ps = [run_robot(robot, n) for robot in robots]
    with open(f'outputs/{n}.txt', 'w+') as f:
        np.savetxt(f, debug_array(np.full((tall, wide), 'O'), ps, False), fmt='%s')
    return get_danger(100)[0], n

day14(101, 103)

(231852216, 8159)

In [ ]:
def day15(flag=False):
    text = data(15)  
    split = text.index('')
    grid, instructions = np.array([list(x) for x in text[:split]]), ''.join(text[split+1:])
    
    robot = icoord(*np.argwhere(grid == '@')[0])
    walls = set()
    for x in np.argwhere(grid == '#'):
        c = icoord(*x)
        walls.add(c)
        if flag:
            walls.add(c+0.5)
    class Box:
        def __init__(self, p):
            self.p = p
        def move(self, diff):
            self.p = tuple([x+diff for x in self.p])
        def ps(self): 
            return self.p
        def __repr__(self):
            return str(self.p)
    boxes = {}
    for x in np.argwhere(grid == 'O'):
        c = icoord(*x)
        box = Box((c, c+0.5) if flag else (c,))
        for p in box.ps():
            boxes[p] = box

    imap = {'^': -1j, 'v': 1j, '>': 0.5 if flag else 1, '<': -0.5 if flag else -1}
    
    def _push(old, diff):
        new = old+diff
        if new in walls:
            return
        touched = set()
        box = boxes.get(new)
        if box:
            touched.add(box)
            for p in box.ps():
                if p == old:
                    continue
                if (t := _push(p, diff)) == None:
                    return
                touched.update(t)
        return touched

    def push(robot, instruction):
        diff = imap[instruction]
        touched = _push(robot, diff)
        if touched is None:
            return robot
        for box in touched:
            for p in box.ps():
                del boxes[p]
        for box in touched:
            box.move(diff)
            for p in box.ps():
                boxes[p] = box
        return robot + diff

    for i in instructions:
        robot = push(robot, i)

    score = 0
    for box in set(boxes.values()):
        ps = [(p.imag, p.real) for p in box.ps()]
        score += 100*min(p[0] for p in ps) + (2 if flag else 1)*min(p[1] for p in ps)
    return int(score)
    
day15(), day15(True)

(1442192, 1448458)

In [ ]:
def day16():
    grid = np.array(data(16, list))
    start = tuple(np.argwhere(grid == 'S')[0])
    end = tuple(np.argwhere(grid == 'E')[0])
    walls = {tuple(x) for x in np.argwhere(grid=='#')}

    # Not using complex numbers because of str ambiguities: eg -0+1j vs 0+1j
    def tuple_add(a, b):
        return (a[0]+b[0], a[1]+b[1])
    def tuple_sub(a, b):
        return (a[0]-b[0], a[1]-b[1])
    def rotate90(a):
        if abs(a[0]):
            yield (0, 1)
            yield (0, -1)
        else:
            yield (1, 0)
            yield (-1, 0)

    def get_neighbors(node, distance):
        pos, dir = node
        new = tuple_add(pos, dir)
        if new not in walls:
            yield (distance+1, (new, dir))
        for d in rotate90(dir):
            yield (distance+1000, (pos, d))
    def end_condition(node, _):
        return node[0] == end
    visited, current = search((start, (0, 1)), get_neighbors, end_condition, method='dijkstra')
    score = visited[current]

    def get_reverse_neighbors(node, _):
        p, dir = node
        distance = visited[(p, dir)]
        new = (tuple_sub(p, dir), dir)
        if visited.get(new, math.inf) == distance-1:
            yield new
        for d in rotate90(dir):
            new = (p, d)
            if visited.get(new, math.inf) == distance-1000:
                yield new
    def start_condition(node, _):
        return node == (start, (0, 1))
    history, _ = search(current, get_reverse_neighbors, start_condition)

    return score, len({x[0] for x in history})

day16()

(85480, 518)

In [195]:
def day17():
    a, b, c, _, program = data(17, lambda x: [int(n) if n else '' for n in (x.split(': ')[-1].split(','))])
    r = {
        4: a[0],
        5: b[0],
        6: c[0],
    }

    def run(r, program):
        def operand(x):
            return r.get(x, x)
        def opcode(c, x):
            match c:
                case 0:
                    r[4] = int(r[4]/(2**operand(x)))
                case 1: 
                    r[5] = r[5]^x
                case 2:
                    r[5] = operand(x)%8
                case 3:
                    return x if r[4] else None
                case 4:
                    r[5] = r[5] ^ r[6]
                case 5:
                    output.append(operand(x)%8)
                case 6:
                    r[5] = int(r[4]/(2**operand(x)))
                case 7:
                    r[6] = int(r[4]/(2**operand(x)))

        p, output = 0, []
        while p < len(program):
            c, x = program[p:p+2]
            jump = opcode(c, x)
            p = jump if jump != None else p+2
        return output, r

    output, _ = run(r, program)

    program = list(reversed([2,4,1,3,7,5,0,3,1,5,4,4,5,5,3,0]))
    def reverse_output(target, bounds):
        for a in range(*bounds):
            # Hardcoded
            b = (a%8)^3
            c = int(a/(2**b))
            b = ((b^5) ^ c) % 8
            if b == target:
                yield a
    def _reverse_input(i, a):
        if i == len(program):
            yield set(a)
        for n in a:
            new = reverse_output(program[i], (n*8, (n+1)*8))
            yield from _reverse_input(i+1, new)
    def reverse_input():
        a = reverse_output(program[0], (0, 7))
        results = list(set(itertools.chain.from_iterable(_reverse_input(1, a))))
        return min(results)

    return ','.join([str(x) for x in output]), reverse_input()

day17()

('6,0,6,3,0,2,3,1,6', 236539226447469)

In [51]:
def day18(size=70):
    corruptions = data(18, lambda x: [int(n) for n in x.split(',')])
    _grid = np.zeros((size+1, size+1))
    end = size+1j*size

    def shortest_path(grid):
        def get_neighbors(node, _):
            for v in (1, -1, 1j, -1j):
                new = node + v
                y, x = yx(new)
                if not (y >= 0 and x >= 0 and y <= size and x <= size):
                    continue
                if not grid[y, x]:
                    yield new
        def end_condition(node, _):
            return node == end
        
        visited, _ = search(0, get_neighbors, end_condition)
        return visited.get(end)

    def corrupted_path(time):
        grid = _grid.copy()
        for i in range(time):
            x, y = corruptions[i]
            grid[y, x] = 1
        return shortest_path(grid), grid
    
    part1, _ = corrupted_path(1024)

    def target(x):
        n, grid = corrupted_path(x)
        if not n:
            return 1
        x, y = corruptions[x+1]
        grid[y, x] = 1
        if not shortest_path(grid):
            return 0
        return -1

    time = binary_search(target, 1024, len(corruptions)-2, 0)
    x, y = corruptions[time]
    return part1, f'{x},{y}'

day18()

(268, '64,11')

In [44]:
def day19():
    text = data(19)
    patterns = text[0].split(', ')
    combinations = text[2:]

    @cache
    def compose(c):
        if not c:
            return 1
        count = 0
        for p in patterns:
            if c.startswith(p):
                count += compose(c[len(p):])
        return count
    
    results = list(filter(lambda x: x, [compose(c) for c in combinations]))
    return len(results), sum(results)

day19()

(317, 883443544805484)

In [99]:
def day20():
    grid = np.array(data(20, list))
    ymax, xmax = grid.shape
    start = icoord(*np.argwhere(grid == 'S')[0])
    end = icoord(*np.argwhere(grid == 'E')[0])
    walls = {icoord(*x) for x in np.argwhere(grid=='#')}

    def get_neighbors(node, _):
        for v in (1, -1, 1j, -1j):
            new = node + v
            y, x = yx(new)
            if not (y >= 0 and x >= 0 and y < ymax and x < xmax):
                continue
            if new not in walls:
                yield new
    def end_condition(node, _):
        return node == end
    
    visited, _ = search(start, get_neighbors, end_condition)
    
    def get_cheats(n):
        cheats = {}
        def get_cheats(node, n):
            for quadrant in [(1, 1j), (-1, 1j), (1, -1j), (-1, -1j)]:
                for i in range(2, n+1):
                    for v in itertools.combinations_with_replacement(quadrant, i):
                        new = node + sum(v)
                        y, x = yx(new)
                        if not (y >= 0 and x >= 0 and y < ymax and x < xmax):
                            continue
                        if new not in walls:
                            saved = visited[new]-visited[node]-i
                            if saved >= 100:
                                cheats[(node, new)] = saved
        
        for node in visited:
            get_cheats(node, n)
        return cheats

    def get_savings(cheats):
        savings = Counter(cheats.values())
        return sum([savings[x] for x in savings])
    
    return get_savings(get_cheats(2)), get_savings(get_cheats(20))

day20()

(1404, 1010981)

In [27]:
def day21():
    codes = data(21)

    def arrow_map():
        g = nx.DiGraph()
        g.add_nodes_from('<>v^A')
        rights = [('<', 'v'), ('v', '>'), ('^', 'A')]
        lefts = [(x[1], x[0]) for x in rights]
        ups = [('v', '^'), ('>', 'A')]
        downs = [(x[1], x[0]) for x in ups]
        g.add_edges_from(rights, d='>')
        g.add_edges_from(ups, d='^')
        g.add_edges_from(lefts, d='<')
        g.add_edges_from(downs, d='v')
        ds = nx.get_edge_attributes(g, 'd')
        return g, ds
    def num_map():
        g = nx.DiGraph()
        for i in range(10):
            g.add_node(str(i))
        g.add_node('A')
        rights = [('0', 'A'), ('1', '2'), ('2', '3'), ('4', '5'), ('5', '6'), ('7', '8'), ('8', '9')]
        lefts = [(x[1], x[0]) for x in rights]
        ups = [('0', '2'), ('A',  '3'), ('1', '4'), ('2', '5'), ('3', '6'), ('4', '7'), ('5', '8'), ('6', '9')]
        downs = [(x[1], x[0]) for x in ups]
        g.add_edges_from(rights, d='>')
        g.add_edges_from(ups, d='^')
        g.add_edges_from(lefts, d='<')
        g.add_edges_from(downs, d='v')
        ds = nx.get_edge_attributes(g, 'd')
        return g, ds
    
    nmap = num_map()
    amap = arrow_map()

    def is_num(i):
        return 47 < ord(i) < 58
    def _get_paths(i, ginfo):
        a, b = i
        g, ds = ginfo
        paths = list(nx.all_shortest_paths(g, a, b))
        return [''.join([ds[(path[i], path[i+1])] for i in range(len(path)-1)])+'A' for path in paths]
    @lru_cache(maxsize=256)
    def get_paths(i):
        if is_num(i[0]) or is_num(i[1]):
            return _get_paths(i, nmap)
        return _get_paths(i, amap)

    def get_path_sequences(code, start='A') :
        if not code:
            return [[]]
        for path in get_paths(start+code[0]):
            sequences = list(get_path_sequences(code[1:], start=code[0]))
            if not sequences:
                yield [path]
            for sequence in sequences:
                yield [path, *sequence]

    @cache
    def press_code(code, n):
        sequences = list(get_path_sequences(code))
        if not n:
            return min(len(''.join(sequence)) for sequence in sequences)
        return min(sum([press_code(c, n-1) for c in sequence]) for sequence in sequences)

    def sum_codes(codes, n):
        return sum([int(code[:-1])*press_code(code, n) for code in codes])

    return sum_codes(codes, 2), sum_codes(codes, 25)

day21()

(248108, 303836969158972)

In [ ]:
# slow: 20s
def day22():
    starts = data(22, int)

    def mixprune(n, s):
        return (n^s)%16777216
    def process(s):
        s = mixprune(s, s*64)
        s = mixprune(s, int(s/32))
        return mixprune(s, s*2048)
    def run(s, n):
        for _ in range(n):
            s = process(s)
            yield s%10
        yield s

    final, c = 0, Counter()
    for start in starts:
        values = {}
        *prices, last = np.array(list(run(start, 2000)))
        final += last
        for i,v in enumerate(np.lib.stride_tricks.sliding_window_view(np.diff(prices), 4)):
            key = tuple(v)
            if key not in values:
                values[key] = prices[i+4]
        c += values
    key = max(c, key=c.get)
    return int(final), int(c[key])


day22()

(18317943467, 2018)

In [20]:
def day22():
    starts = data(22, int)

    def mixprune(n, s):
        return (n^s)%16777216
    def process(s):
        s = mixprune(s, s*64)
        s = mixprune(s, int(s/32))
        return mixprune(s, s*2048)
    def run(s, n):
        for _ in range(n):
            s = process(s)
            yield s

    final, counts = 0, Counter()
    for start in starts:
        seen = set()
        secrets = run(start, 2000)
        i = [next(secrets)%10 for _ in range(4)]
        previous, diffs = i[-1], (0, i[1]-i[0], i[2]-i[1], i[3]-i[2])
        for secret in secrets:
            price = secret%10
            diffs, previous = (diffs[1], diffs[2], diffs[3], price-previous), price
            if diffs not in seen:
                counts[diffs] += price
                seen.add(diffs)
        final += secret

    key = max(counts, key=counts.get)
    return int(final), int(counts[key])

day22()

(18317943467, 2018)

In [65]:
def day23():
    connections = data(23, lambda x: x.split('-'))
    g = nx.Graph()
    g.add_edges_from(connections)

    def starts_t(x):
        return any([c for c in x if c.startswith('t')])
    
    cliques = list(nx.enumerate_all_cliques(g))
    part1 = len([x for x in cliques if len(x) == 3 and starts_t(x)])
    part2 = ','.join(sorted(max(cliques, key=len)))
    return part1, part2    

day23()

(1083, 'as,bu,cp,dj,ez,fd,hu,it,kj,nx,pp,xh,yu')

In [ ]:
def day24():
    text = data(0)
    split = text.index('')
    vals = {y[0]:int(y[1]) for y in (x.split(': ') for x in text[:split])}

    class Gate:
        def __init__(self, text):
            self.a, self.op, self.b, self.c = text
            self.children = set()
        def add_child(self, child):
            self.children.add(child)
        def __repr__(self):
            return f'{self.a} {self.op} {self.b} -> {self.c}'

    gates = {gate.c:gate for gate in (Gate(re.findall(r'(...) (AND|OR|XOR) (...) -> (...)', x)[0]) for x in text[split+1:])} 

    

In [192]:
def day24():
    text = data(0)
    split = text.index('')
    vals = {y[0]:int(y[1]) for y in (x.split(': ') for x in text[:split])}

    def bits(l, values):
        return ''.join((str(values[k])) for k in (sorted([x for x in values if x.startswith(l)], reverse=True)))

    class Gate:
        def __init__(self, text):
            self.a, self.op, self.b, self.c = text
            self.children = set()
        def add_child(self, child):
            self.children.add(child)
        def __repr__(self):
            return f'{self.a} {self.op} {self.b} -> {self.c}'

    gates = {gate.c:gate for gate in (Gate(re.findall(r'(...) (AND|OR|XOR) (...) -> (...)', x)[0]) for x in text[split+1:])}    
    start = deque()
    for c in gates:
        gate = gates[c]            
        if gate.a[0] in ('x', 'y') and gate.b[0] in ('x', 'y'):
            start.append(c)
        if gate.a in gates:
            gates[gate.a].add_child(gate)
        if gate.b in gates:
            gates[gate.b].add_child(gate)

    def apply(q, values, swaps):
        def run(c):
            gate = gates[c]
            a, op, b = gate.a, gate.op, gate.b
            match op:
                case 'AND':
                    result = values[a] and values[b]
                case 'OR':
                    result = values[a] or values[b]
                case 'XOR':
                    result = values[a] ^ values[b]
            c = swaps.get(c, c)
            values[c] = result
            return gates[c]

        visited = set()
        while q:
            c = q.popleft()
            visited.add(c)
            gate = run(c)
            for child in gate.children:
                if child.a in values and child.b in values:
                    if child.c not in visited:
                     q.append(child.c)
        return bits('z', values)

    def gen_values(n):
        def _gen_values():
            values = vals.copy()
            for i in values:
                values[i] = random.randint(0, 1)
            return values
        for _ in range(n):
            values = _gen_values()
            yield values

    def similarity(a, b):
        return sum(a[i]==b[i] for i in range(len(a)))

    def target_fn(xs, ys, fill=6):
        part = int(xs, 2)&int(ys, 2)
        # part = int(xs, 2) + int(ys, 2)
        return str(bin(part))[2:].zfill(fill)
    
    # print(len(list((itertools.combinations_with_replacement(gates, 4)))))

    swap_scores = {x: 0 for x in (itertools.combinations(gates, 2))}
    for values in gen_values(10):
        xs = bits('x', values)
        ys = bits('y', values)
        noswap = apply(start.copy(), values, {})
        expected = target_fn(xs, ys)
        baseline = similarity(noswap, expected)
        # print('x', xs, 'y', ys, 'z', expected, noswap, baseline)
        for swap in swap_scores:
            if swap_scores[swap] < 0:
                continue
            swaps = {swap[0]: swap[1], swap[1]: swap[0]}
            result = apply(start.copy(), values, swaps)
            score = similarity(result, expected)
            swap_scores[swap] += score - baseline
            # print(swap, result, score)

    n = 2
    top_swaps = sorted(swap_scores.items(), key=lambda item: item[1], reverse=True)
    print(top_swaps)

    def check():
        for pairings in itertools.combinations([x[0] for x in top_swaps], 4):
            
            swaps = {}
            for i in pairings:
                print('a', i)
                swaps[i[0]], swaps[i[1]] = i[1], i[0]
            print('wtf', pairings, swaps, len(swaps))
            for values in gen_values(2):
                xs = bits('x', values)
                ys = bits('y', values)
                noswap = apply(start.copy(), values, {})
                expected = target_fn(xs, ys)
                baseline = similarity(noswap, expected)
                result = apply(start.copy(), values, swaps)
                score = similarity(result, expected)
                if result == expected:
                    print('x', xs, 'y', ys, 'z', expected, noswap, baseline)
                    print(swaps, result, score)
                    return swaps

    test = check()
    print(test)

    # print(top_swaps)
    print(','.join(sorted(flatten(dict(top_swaps[:n]).keys()))))

    # return int(baseline, 2)

day24()

[(('z05', 'z00'), 12), (('z02', 'z01'), 12), (('z05', 'z02'), 2), (('z05', 'z01'), 2), (('z02', 'z00'), 2), (('z01', 'z00'), 2), (('z05', 'z04'), 0), (('z04', 'z00'), 0), (('z05', 'z03'), -2), (('z02', 'z03'), -2), (('z02', 'z04'), -2), (('z01', 'z03'), -2), (('z01', 'z04'), -2), (('z03', 'z04'), -2), (('z03', 'z00'), -2)]
a ('z05', 'z00')
a ('z02', 'z01')
a ('z05', 'z02')
a ('z05', 'z01')
wtf (('z05', 'z00'), ('z02', 'z01'), ('z05', 'z02'), ('z05', 'z01')) {'z05': 'z01', 'z00': 'z05', 'z02': 'z05', 'z01': 'z05'} 4
a ('z05', 'z00')
a ('z02', 'z01')
a ('z05', 'z02')
a ('z02', 'z00')
wtf (('z05', 'z00'), ('z02', 'z01'), ('z05', 'z02'), ('z02', 'z00')) {'z05': 'z02', 'z00': 'z02', 'z02': 'z00', 'z01': 'z02'} 4
a ('z05', 'z00')
a ('z02', 'z01')
a ('z05', 'z02')
a ('z01', 'z00')
wtf (('z05', 'z00'), ('z02', 'z01'), ('z05', 'z02'), ('z01', 'z00')) {'z05': 'z02', 'z00': 'z01', 'z02': 'z05', 'z01': 'z00'} 4
x 001011 y 110000 z 000000 000000 6
{'z05': 'z02', 'z00': 'z01', 'z02': 'z05', 'z01': '

In [193]:
def day24():
    text = data(24)
    split = text.index('')
    vals = {y[0]:int(y[1]) for y in (x.split(': ') for x in text[:split])}

    def bits(l, values):
        return ''.join((str(values[k])) for k in (sorted([x for x in values if x.startswith(l)], reverse=True)))

    class Gate:
        def __init__(self, text):
            self.a, self.op, self.b, self.c = text
            self.children = set()
        def add_child(self, child):
            self.children.add(child)
        def __repr__(self):
            return f'{self.a} {self.op} {self.b} -> {self.c}'

    gates = {gate.c:gate for gate in (Gate(re.findall(r'(...) (AND|OR|XOR) (...) -> (...)', x)[0]) for x in text[split+1:])}    
    start = deque()
    for c in gates:
        gate = gates[c]            
        if gate.a[0] in ('x', 'y') and gate.b[0] in ('x', 'y'):
            start.append(c)
        if gate.a in gates:
            gates[gate.a].add_child(gate)
        if gate.b in gates:
            gates[gate.b].add_child(gate)

    def apply(q, values, swaps):
        def run(c):
            gate = gates[c]
            a, op, b = gate.a, gate.op, gate.b
            match op:
                case 'AND':
                    result = values[a] and values[b]
                case 'OR':
                    result = values[a] or values[b]
                case 'XOR':
                    result = values[a] ^ values[b]
            c = swaps.get(c, c)
            values[c] = result
            return gates[c]

        visited = set()
        while q:
            c = q.popleft()
            visited.add(c)
            gate = run(c)
            for child in gate.children:
                if child.a in values and child.b in values:
                    if child.c not in visited:
                     q.append(child.c)
        return bits('z', values)

    def gen_values(n):
        def _gen_values():
            values = vals.copy()
            for i in values:
                values[i] = random.randint(0, 1)
            return values
        for _ in range(n):
            values = _gen_values()
            yield values

    def similarity(a, b):
        return sum(a[i]==b[i] for i in range(len(a)))

    def target_fn(xs, ys, fill=6):
        # part = int(xs, 2)&int(ys, 2)
        part = int(xs, 2) + int(ys, 2)
        return str(bin(part))[2:].zfill(fill)

    swap_scores = {x: 0 for x in (itertools.combinations(gates, 2))}
    for i, values in enumerate(gen_values(2)):
        xs = bits('x', values)
        ys = bits('y', values)
        noswap = apply(start.copy(), values, {})
        expected = target_fn(xs, ys, 46)
        baseline = similarity(noswap, expected)
        # print('x', xs, 'y', ys, 'z', expected, noswap, baseline)
        for swap in swap_scores:
            if i > 3 and swap_scores[swap] == 0:
                continue
            swaps = {swap[0]: swap[1], swap[1]: swap[0]}
            result = apply(start.copy(), values, swaps)
            score = similarity(result, expected)
            swap_scores[swap] += score - baseline
            # print(swap, result, score)

    # n = 4 #2
    top_swaps = sorted(swap_scores.items(), key=lambda item: abs(item[1]), reverse=True)
    print('t', top_swaps)

    def check():
        for pairings in itertools.combinations([x[0] for x in top_swaps[:25]], 4):
            swaps = {}
            for i in pairings:
                swaps[i[0]], swaps[i[1]] = i[1], i[0]
            if len(swaps) < 8:
                continue
            flag = False
            for values in gen_values(3):
                xs = bits('x', values)
                ys = bits('y', values)
                noswap = apply(start.copy(), values, {})
                expected = target_fn(xs, ys, 46)
                baseline = similarity(noswap, expected)
                result = apply(start.copy(), values, swaps)
                score = similarity(result, expected)
                if result == expected:
                    # print('x', xs, 'y', ys, 'z', expected, noswap, baseline)
                    # print(swaps, result, score)
                    if flag:
                        return pairings
                    flag = True

    result = check()
    print(result)
    print(','.join(sorted(flatten(result))))

    # return int(baseline, 2)

day24()

KeyboardInterrupt: 

In [191]:
def day25():
    text = data(24)
    split = text.index('')
    vals = {y[0]:int(y[1]) for y in (x.split(': ') for x in text[:split])}

    def bits(l, values):
        return ''.join((str(values[k])) for k in (sorted([x for x in values if x.startswith(l)], reverse=True)))

    class Gate:
        def __init__(self, text):
            self.a, self.op, self.b, self.c = text
            self.children = set()
        def add_child(self, child):
            self.children.add(child)
        def __repr__(self):
            return f'{self.a} {self.op} {self.b} -> {self.c}'

    gates = {gate.c:gate for gate in (Gate(re.findall(r'(...) (AND|OR|XOR) (...) -> (...)', x)[0]) for x in text[split+1:])}    
    start = deque()
    for c in gates:
        gate = gates[c]            
        if gate.a[0] in ('x', 'y') and gate.b[0] in ('x', 'y'):
            start.append(c)
        if gate.a in gates:
            gates[gate.a].add_child(gate)
        if gate.b in gates:
            gates[gate.b].add_child(gate)

    def apply(q, values, swaps):
        def run(c):
            gate = gates[c]
            a, op, b = gate.a, gate.op, gate.b
            match op:
                case 'AND':
                    result = values[a] and values[b]
                case 'OR':
                    result = values[a] or values[b]
                case 'XOR':
                    result = values[a] ^ values[b]
            c = swaps.get(c, c)
            values[c] = result
            return gates[c]

        visited = set()
        while q:
            c = q.popleft()
            visited.add(c)
            gate = run(c)
            for child in gate.children:
                if child.a in values and child.b in values:
                    if child.c not in visited:
                     q.append(child.c)
        return bits('z', values)

    def gen_values(n):
        def _gen_values():
            values = vals.copy()
            for i in values:
                values[i] = random.randint(0, 1)
            return values
        for _ in range(n):
            values = _gen_values()
            yield values

    def similarity(a, b):
        return sum(a[i]==b[i] for i in range(len(a)))

    def target_fn(xs, ys, fill=6):
        # part = int(xs, 2)&int(ys, 2)
        part = int(xs, 2) + int(ys, 2)
        return str(bin(part))[2:].zfill(fill)

    swap_scores = {x: 0 for x in (itertools.combinations(gates, 2))}
    for i, values in enumerate(gen_values(10)):
        xs = bits('x', values)
        ys = bits('y', values)
        noswap = apply(start.copy(), values, {})
        expected = target_fn(xs, ys, 46)
        baseline = similarity(noswap, expected)
        print('x', xs, 'y', ys, 'z', expected, noswap, baseline)
        for swap in swap_scores:
            if i > 3 and swap_scores[swap] == 0:
                continue
            swaps = {swap[0]: swap[1], swap[1]: swap[0]}
            result = apply(start.copy(), values, swaps)
            score = similarity(result, expected)
            swap_scores[swap] += score - baseline
            # print(swap, result, score)
    print(swap_scores)

    test = list(itertools.combinations([x for x in swap_scores if abs(swap_scores[x])>20], 2))
    print(len(test))

    pair_scores = Counter()
    for pairings in test:
        swaps = {}
        for i in pairings:
            swaps[i[0]], swaps[i[1]] = i[1], i[0]
        if len(swaps) < 4:
            continue
        for i, values in enumerate(gen_values(2)):
            xs = bits('x', values)
            ys = bits('y', values)
            noswap = apply(start.copy(), values, {})
            expected = target_fn(xs, ys, 46)
            baseline = similarity(noswap, expected)
            result = apply(start.copy(), values, swaps)
            score = similarity(result, expected)
            pair_scores[swap] += score - baseline
    print(pair_scores)

    # # n = 4 #2
    # top_swaps = sorted(swap_scores.items(), key=lambda item: abs(item[1]), reverse=True)
    # print('t', top_swaps)

    # def check():
    #     for pairings in itertools.combinations([x[0] for x in top_swaps[:25]], 4):
    #         swaps = {}
    #         for i in pairings:
    #             swaps[i[0]], swaps[i[1]] = i[1], i[0]
    #         if len(swaps) < 8:
    #             continue
    #         flag = False
    #         for values in gen_values(3):
    #             xs = bits('x', values)
    #             ys = bits('y', values)
    #             noswap = apply(start.copy(), values, {})
    #             expected = target_fn(xs, ys, 46)
    #             baseline = similarity(noswap, expected)
    #             result = apply(start.copy(), values, swaps)
    #             score = similarity(result, expected)
    #             if result == expected:
    #                 # print('x', xs, 'y', ys, 'z', expected, noswap, baseline)
    #                 # print(swaps, result, score)
    #                 if flag:
    #                     return pairings
    #                 flag = True

    # result = check()
    # print(result)
    # print(','.join(sorted(flatten(result))))

    # return int(baseline, 2)

day25()

x 110010010010010110010000111110011101011010000 y 101100100100000101101100110001111010001011001 z 1011110110110011011111101110000010111100101001 1011110110110011100111101010000010111100101001 42
x 010111010010010011000000011011011101001011010 y 010000010110011111011000101110110100011001010 z 0100111101000110010011001001010010001100100100 0100111101000110001011000101010010001100100100 42
x 010010110110000011011100110011110110111000010 y 001001000100100111101001100010100001001000111 z 0011011111010101011000110010110011000000001001 0011011111010101010000110010110011000000001001 45
x 100110101100001100000111001111001010111001011 y 100101110110100001011111110111111000110010110 z 1001100100010101101100111000111000011101100001 1001100100010101110100110100111000011101100001 42
x 100111001100101001101000101111011100110010101 y 111100001000001000011010000110110100101000100 z 1100011010100110010000010110110010001011011001 1100011010100110011000011010110010001011011001 43
x 11101010110101101001011

KeyboardInterrupt: 

In [175]:
a = (('stm', 'njd'), ('qck', 'z27'), ('smf', 'jkk'), ('rfd', 'mpw'))
flatten(a)

['stm', 'njd', 'qck', 'z27', 'smf', 'jkk', 'rfd', 'mpw']

In [182]:
len(list(itertools.combinations(range(25), 4)))

12650